In [1]:
!pip install pytket

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 12.3 MB/s eta 0:00:00


In [2]:
!pip install pytket-quantinuum
!pip install pytket-qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 10.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pyjwt[crypto] to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6

In [59]:
from numpy.lib.function_base import kaiser
from pytket.utils import probs_from_counts, counts_from_shot_table, expectation_from_counts

import numpy as np

# We use this paper to reconstruct our density matrix by IBM: https://arxiv.org/pdf/1106.5458.pdf
# We will do the fidelity reconstruction over multiple qubits to allow generalization

I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
basis = [I, X, Y, Z]

# Direct computation of expectation using numpy operations
def get_expectation(shot_table):
  aritysum = np.sum(np.sum(shot_table, axis=1) % 2)
  return (-2 * aritysum) / shot_table.shape[0] + 1

# Use pytket library methods to compute expectation from shot table
def pytket_get_expectation(shot_table):
  return expectation_from_counts(counts_from_shot_table(shot_table))

def get_n_qubit_basis_element(index, nqubits):
  n_qubit_basis_element = np.array([1.])
  for qubit_index in range(nqubits):
    mask = 3 << (2 * qubit_index)
    basis_index = (index & mask) >> (2 * qubit_index)
    n_qubit_basis_element = np.kron(n_qubit_basis_element, basis[basis_index])
  return n_qubit_basis_element

def build_mle_matrix(expectations, nqubits):
  dim = 2 ** nqubits
  result = np.zeros((dim, dim))
  for index, expectation in enumerate(expectations):
    result = result + expectation * get_n_qubit_basis_element(index, nqubits)
  return result

# Construct the density matrix given shot data for d^2 Hermitian projectors that span the dxd matrix space
# d = 2^n is the dimension of one side of the matrix
def get_density_matrix(shot_tables, nqubits):
  dim = 2 ** nqubits
  expectations = np.array([pytket_get_expectation(shot_table) for shot_table in shot_tables])
  mle_matrix = build_mle_matrix(expectations, nqubits) / dim
  # print("MLE Matrix: ", mle_matrix)
  eigenvalues, eigenvectors = np.linalg.eig(mle_matrix)
  # print("MLE Matrix Eigenvalues: ", eigenvalues)
  mapped_eigenvalues = [(eigenvalue, i) for i, eigenvalue in enumerate(eigenvalues)]
  mapped_eigenvalues.sort()
  mapped_eigenvalues.reverse()
  # print("Sorted eigenvalues: ", mapped_eigenvalues)
  accumulator = 0
  indicator = -1
  for i in range(len(mapped_eigenvalues)-1, -1, -1):
    eigenvalue, eigenvector_index = mapped_eigenvalues[i]
    k = i + 1
    indicator = eigenvalue + accumulator / k
    mapped_eigenvalues[i] = (max(0, indicator), eigenvector_index)
    if indicator < 0:
      accumulator += eigenvalue
  print("Reduced Eigenvalues: ", mapped_eigenvalues)
  # print("Eigenvectors: ", eigenvectors)
  density_matrix = np.zeros((dim, dim), dtype=np.complex128)
  for eigenvalue, index in mapped_eigenvalues:
    eigenvector = eigenvectors[index].reshape((-1, 1))
    density_matrix += (eigenvector @ np.matrix.getH(eigenvector)) * eigenvalue
  return density_matrix

# Compute squareroot of matrix using diagonalization and squareroot of eigenvalues
def get_sqrt_matrix(density_matrix):
  evalues, evectors = np.linalg.eig(density_matrix)
  sqrt_matrix = (evectors * np.sqrt(evalues)) @ np.linalg.inv(evectors)
  return sqrt_matrix

# We now compute fidelity
def get_fidelity(expected_density_matrix, shot_tables, nqubits):
  density_matrix = get_density_matrix(shot_tables, nqubits)
  sqrt_matrix = get_sqrt_matrix(density_matrix)
  return np.trace(get_sqrt_matrix(sqrt_matrix @ expected_density_matrix @ sqrt_matrix)) ** 2



In [80]:
from pytket import Circuit
from pytket.extensions.qiskit import AerBackend

from pytket import Circuit
from pytket.circuit.display import get_circuit_renderer

circuit_renderer = get_circuit_renderer() # Instantiate a circuit renderer
circuit_renderer.set_render_options(zx_style=True) # Configure render options
circuit_renderer.condense_c_bits = False # You can also set the properties on the instance directly
print("Render options:")
print(circuit_renderer.get_render_options()) # View currently set render options

circuit_renderer.min_height = "300px" # Change the display height

# Circuit should not have performed measurement of ANY kind
def obtain_shots_information(circuit, nqubits, backend, nshots):
  matrix_basis_dim = 4 ** nqubits
  shots_tables = []
  for basis_index in range(matrix_basis_dim):
    base = circuit.copy()
    for qubit_index in range(nqubits):
      mask = 3 << (2 * qubit_index)
      operator_index = (basis_index & mask) >> (2 * qubit_index)

      # I - Discard this qubit in this particular measurement
      # X - Add a Hadamard gate to measure in X basis
      # Y - Add a S dagger and Hadamard to measure in Y basis
      # Z - Do Nothing - Measurement performed in Z basis
      if operator_index == 0:
        continue

      if operator_index == 1:
        base.H(qubit_index)

      if operator_index == 2:
        base.Sdg(qubit_index)
        base.H(qubit_index)

      base.Measure(qubit_index, qubit_index)

    # print("Operator index: ", bin(basis_index))
    # circuit_renderer.render_circuit_jupyter(base)
    compiled_circ = backend.get_compiled_circuit(base)
    handle = backend.process_circuit(compiled_circ, n_shots=nshots)
    shots = backend.get_result(handle).get_shots()
    shots_tables.append(shots)
  return shots_tables

Render options:
{'zx_style': True, 'condense_c_bits': False}


In [88]:
from pytket import Circuit
from pytket.extensions.qiskit import AerBackend

nqubits=3
nshots=10000
circ = Circuit(nqubits, nqubits)
circ.H(0).H(1).X(1).CX(1, 2)
backend = AerBackend()
# compiled_circ = backend.get_compiled_circuit(circ)

# handle = backend.process_circuit(compiled_circ, n_shots=1000)
# shots = backend.get_result(handle).get_shots()

# print(probs_from_counts(counts_from_shot_table(shots)))
# print(pytket_get_expectation(shots))

shots_tables = obtain_shots_information(circ, nqubits, backend, nshots)
matrix = get_density_matrix(shots_tables, nqubits)
print(matrix)

Reduced Eigenvalues:  [((0.97523259367279+4.568137580652041e-19j), 0), ((0.002777944781307263+4.592235610671397e-18j), 1), ((0.0012820253435614128-2.3876859127348835e-18j), 2), (0, 6), (0, 7), (0, 5), (0, 4), (0, 3)]
[[ 0.24954915+1.16905286e-19j  0.1176066 -1.10899483e-01j
   0.31424905-6.52048794e-07j -0.00551559-1.96539125e-02j
  -0.05293879-2.43671174e-02j -0.09665998-9.34645091e-02j
  -0.1647558 -4.87184789e-02j  0.06538683+2.29361353e-02j]
 [ 0.1176066 +1.10899483e-01j  0.1053287 +9.70724470e-19j
   0.14805942+1.39447443e-01j  0.00568581-1.17107855e-02j
  -0.01356746-3.51221558e-02j -0.00415877-8.66441806e-02j
  -0.05613435-9.63010866e-02j  0.01988405+3.98898044e-02j]
 [ 0.31424905+6.52048794e-07j  0.14805942-1.39447443e-01j
   0.39596866+1.57699797e-19j -0.00706894-2.50777999e-02j
  -0.06683486-3.05025819e-02j -0.12175218-1.17779802e-01j
  -0.20712271-6.12740497e-02j  0.08226576+2.86211402e-02j]
 [-0.00551559+1.96539125e-02j  0.00568581+1.17107855e-02j
  -0.00706894+2.50777999e-